In [1]:
!pip install krisi
!pip install plotly, kaleido, statsmodels, weasyprint

ERROR: Invalid requirement: 'plotly,': Expected end or semicolon (after name and no valid version specifier)
    plotly,
          ^


In [3]:
from alphaswarm.services.alchemy import AlchemyClient
from alphaswarm.tools.alchemy.alchemy_price_history import GetAlchemyPriceHistoryBySymbol
from alphaswarm.tools.forecasting.price_forecasting import ForecastTokenPrice
from alphaswarm.services.alchemy import HistoricalPriceBySymbol

import numpy as np
from krisi.evaluate import score

In [4]:
total_days = 130
num_y = 30

alchemy_client = AlchemyClient.from_env()
tool = GetAlchemyPriceHistoryBySymbol(alchemy_client)
gold_result = tool.forward(symbol="WETH", interval="1d", history=total_days)

# use the first 6 days data as input to the forecaster
historical_data = HistoricalPriceBySymbol(symbol=gold_result.symbol, data=[p for p in gold_result.data[:-num_y]])

In [5]:
len(historical_data.data)

100

## Predictions without context

In [7]:
forecaster = ForecastTokenPrice()
forecast_data = forecaster.forward(historical_price_data = historical_data,
                              forecast_horizon=f"everyday for next {num_y} days",
                              supporting_context=None
                              )

In [8]:
references = np.array([float(p.value) for p in gold_result.data[-num_y:]])
predictions_1 = np.array([float(f.price) for f in forecast_data.forecast[1:]])
scorecard_1 = score(y=references, predictions=predictions_1)
scorecard_1.print('minimal')

/Users/jerrychen/Desktop/GitHub/alphaswarm/venv/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Users/jerrychen/Desktop/GitHub/alphaswarm/venv/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Model_20250306-190307ec02338f
                     Mean Absolute Error                   9.066591e+02
          Mean Absolute Percentage Error                   3.762848e-01
Symmetric Mean Absolute Percentage Error                   1.472784e-01
                      Mean Squared Error                   1.182431e+06
                 Root Mean Squared Error                   1.087396e+03
                               R-squared                  -2.597678e+01
                   Mean of the Residuals                  -9.057747e+02
     Standard Deviation of the Residuals                   6.119524e+02

In [9]:
len(forecast_data.forecast)

11

## Try again with additional context

In [29]:
additional_context = """
The value of AIXBT between February 11 and February 17, 2025, was influenced by several key developments and market dynamics:

### Price Predictions and Market Sentiment
- **CryptoPredictions.com's February 2025 Forecast**: Released around this period, the projection anticipated AIXBT closing February at **$0.1924**, with a potential peak of **$0.4011** and a floor of **$0.1924**[2]. This mixed outlook likely caused cautious trading behavior.
- **MEXC's Conservative Estimate**: On February 15, MEXC predicted AIXBT would reach **$0.2180** by February 24 using a 5% annual growth model[3], reinforcing short-term stability expectations.

### Exchange Listings and Volatility
- **Binance Integration**: AIXBT's January 10 listing on Binance (with pairs like AIXBT/USDT) continued to impact liquidity and accessibility during February, contributing to daily volatility[4]. The "Seed Tag" label highlighted its early-stage risks, affecting investor confidence[2].

### Whale Activity and Market Manipulation Concerns
- **Whale Accumulation**: Large holders purchased **3.81M AIXBT** in early February, driving a 41% price surge[6]. By February 17, whale activity remained a focal point, with concerns that concentrated buying could lead to sharp corrections if demand waned[7].

### Technical Analysis and Trading Strategies
- **Support/Resistance Levels**: Traders monitored key levels, including:
  - **Support**: $0.2600–$0.2700
  - **Resistance**: $0.3100–$0.3250[1][7]
- **Bullish/Bearish Setups**: Strategies circulated for long positions (targeting **$0.3100–$0.3200**) and short positions (targeting **$0.2700–$0.2800**)[1], influencing intraweek price swings.

### Sector-Wide Trends
- **AI Token Recovery**: The broader AI agent token sector rebounded from $11B to $14.66B during this period[7], lifting AIXBT as a top performer. Its integration with AI-driven market analytics tools kept it in focus[11].

---

While no major partnerships or protocol updates occurred during this specific window, AIXBT’s price action was shaped by lingering effects of its Binance listing, whale-driven volatility, and evolving sector trends.
"""

In [28]:
additional_context = """
Ethereum's market dynamics in the first 20 days of the last 30 days (January 20–February 8, 2025) were shaped by several key developments in the U.S. regulatory and institutional landscape:  

### U.S. Regulatory Shifts  
- On **January 23**, President Trump issued an executive order revamping crypto policy to prioritize blockchain innovation and oppose central bank digital currencies (CBDCs). This directive established a federal working group to redefine crypto regulations and evaluate a national crypto asset reserve[8].  

### Institutional Activity  
- **Goldman Sachs** and **BlackRock** signaled growing interest in Ethereum, with reports of potential acquisitions for a U.S. crypto reserve[9].  
- **World Liberty Financial** (WLF), linked to the Trump family, continued large-scale ETH purchases, aligning with its strategy to bolster institutional crypto holdings[6][9].  

### ETF Inflows  
- Spot Ethereum ETFs saw notable inflows, with **BlackRock** and **Fidelity** leading institutional demand. By February 3, Ethereum ETFs had absorbed 145,000 ETH, reflecting heightened investor confidence despite market volatility[12].  

### On-Chain Metrics  
- Ethereum’s network activity surged, with daily active addresses peaking at **712,276** on January 25—a nine-month high. Transactional activity also rose, signaling renewed user engagement[6].  

These developments underscore Ethereum’s evolving role in U.S. financial infrastructure, driven by regulatory clarity, institutional adoption, and network growth.
"""

forecast_data_2 = forecaster.forward(historical_price_data = historical_data,
                              forecast_horizon="everyday for next 10 days",
                              supporting_context=additional_context
                              )

In [26]:
forecast_data_2.forecast

[PriceForecast(timestamp=datetime.datetime(2025, 2, 9, 0, 0, tzinfo=TzInfo(UTC)), price=Decimal('2635.4'), lower_confidence_bound=Decimal('2635.4'), upper_confidence_bound=Decimal('2635.4')),
 PriceForecast(timestamp=datetime.datetime(2025, 2, 10, 0, 0, tzinfo=TzInfo(UTC)), price=Decimal('2680'), lower_confidence_bound=Decimal('2590'), upper_confidence_bound=Decimal('2770')),
 PriceForecast(timestamp=datetime.datetime(2025, 2, 11, 0, 0, tzinfo=TzInfo(UTC)), price=Decimal('2740'), lower_confidence_bound=Decimal('2620'), upper_confidence_bound=Decimal('2860')),
 PriceForecast(timestamp=datetime.datetime(2025, 2, 12, 0, 0, tzinfo=TzInfo(UTC)), price=Decimal('2815'), lower_confidence_bound=Decimal('2650'), upper_confidence_bound=Decimal('2980')),
 PriceForecast(timestamp=datetime.datetime(2025, 2, 13, 0, 0, tzinfo=TzInfo(UTC)), price=Decimal('2900'), lower_confidence_bound=Decimal('2680'), upper_confidence_bound=Decimal('3120')),
 PriceForecast(timestamp=datetime.datetime(2025, 2, 14, 0, 0

In [30]:
len(forecast_data_2.forecast)

10

In [32]:
predictions_2 = np.array([float(f.price) for f in forecast_data_2.forecast[1:]])
scorecard_2 = score(y=references, predictions=predictions_2)
scorecard_2.print('minimal')

AssertionError: Target length 10 should match predictions length 9

## Comparing

In [21]:
from krisi import compare

compare([scorecard_1, scorecard_2], sort_by="r_two")

,r_two,mae,mape,smape,residuals_mean,residuals_std
Model_20250219-155842b9ae7a75,-13.221972,142.483792,0.052803,0.027318,142.483792,78.721074
Model_20250219-155511f1fa3797,-19.654719,166.983792,0.061893,0.032293,166.983792,103.819638


## Saving and Loading

In [111]:
scorecard_1.metadata.project_name = "no context predictions - aixbt - 0218"
scorecard_1.save(save_modes=["obj"])

<krisi.evaluate.scorecard.ScoreCard object at 0x13fdf6de0
           Model_20250218-182932253fc0f0 | Dataset_20250218-18293219650564 
   no context predictions - aixbt - 0218 | regression

                     Mean Absolute Error - 0.0056487      
          Mean Absolute Percentage Error - 0.031038       
Symmetric Mean Absolute Percentage Error - 0.015289       
                               R-squared - 0.5489         
                   Mean of the Residuals - -0.0025328     
     Standard Deviation of the Residuals - 0.0062166      >

In [125]:
scorecard_2.metadata.project_name = "with context predictions - aixbt - 0218"
scorecard_2.save(save_modes=["obj"])

<krisi.evaluate.scorecard.ScoreCard object at 0x13fcf28a0
           Model_20250218-18284890d0c75b | Dataset_20250218-182848e8eb26fc 
 with context predictions - aixbt - 0218 | regression

                     Mean Absolute Error - 0.005482       
          Mean Absolute Percentage Error - 0.029957       
Symmetric Mean Absolute Percentage Error - 0.01483        
                               R-squared - 0.59111        
                   Mean of the Residuals - -0.0015328     
     Standard Deviation of the Residuals - 0.0062166      >

In [1]:

from krisi import load_scorecards

last_scorecard = load_scorecards("no context predictions - aixbt - 0218")[-1]
last_scorecard.print("minimal")
last_scorecard.generate_report('pdf')

Model_20250218-182932253fc0f0
                     Mean Absolute Error                       0.005649
          Mean Absolute Percentage Error                       0.031038
Symmetric Mean Absolute Percentage Error                       0.015289
                               R-squared                       0.548901
                   Mean of the Residuals                      -0.002533
     Standard Deviation of the Residuals                       0.006217


-----

WeasyPrint could not import some external libraries. Please carefully follow the installation steps before reporting an issue:
https://doc.courtbouillon.org/weasyprint/stable/first_steps.html#installation
https://doc.courtbouillon.org/weasyprint/stable/first_steps.html#troubleshooting 

-----



OSError: cannot load library 'libgobject-2.0-0': dlopen(libgobject-2.0-0, 0x0002): tried: 'libgobject-2.0-0' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibgobject-2.0-0' (no such file), '/usr/lib/libgobject-2.0-0' (no such file, not in dyld cache), 'libgobject-2.0-0' (no such file).  Additionally, ctypes.util.find_library() did not manage to locate a library called 'libgobject-2.0-0'